In [28]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
cd ../TileEM/

/home/jlee782/crowd-seg/analysis/TileEM


In [10]:
cd ../pixel_em/

/home/jlee782/crowd-seg/analysis/TileEM/pixel_em


In [12]:
import pandas as pd

In [13]:
PR_tbl = pd.read_csv("full_PR_table.csv")

In [15]:
PR_tbl.mean()

num_workers     12.333333
sample_num       2.933333
objid           24.000000
MV_precision     0.923356
MV_recall        0.927232
EM_precision     0.922656
EM_recall        0.928828
dtype: float64